In [ ]:
"""
Super-7 Vertex AI Extractor (Candidate Entity Version, Clean Output)

Features:
- Uses Vertex AI Gemini with Google Search grounding
- 3-attempt retry engine with different search strategies + early stop
- LLM extracts clean Super-7 + per-field source metadata
- Candidate-entity layer:
    - Uses input name + discovered names as separate candidates
    - Runs full extraction (all strategies) for each candidate name
    - Scores candidates (name similarity + website/address + model confidence)
    - Chooses the best candidate as final result
- Clean JSON output per company:
    {
      "input": {...original CSV row...},
      "super7": {...final chosen values...},
      "sources": {
         "company_name": {
            "source_url": "...",
            "source_type": "...",
            "rationale": "..."
         },
         ...
      },
      "candidate_debug": [
         {"candidate_name": "...", "score": 0.83},
         ...
      ]
    }
"""

import json
import re
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple

import pandas as pd
from google import genai
from google.genai import types


# ==================== CONFIG ====================

PROJECT_ID = "YOUR_GCP_PROJECT_ID"  # <-- set this
LOCATION = "us-east4"
MODEL_NAME = "gemini-2.0-flash"

MAX_RETRIES = 3
EARLY_STOP_THRESHOLD = 0.92  # early-stop if attempt score >= this

# Expected CSV columns (you can have more; we just use these as hints)
DEFAULT_INPUT_COLS = [
    "company_name",
    "country",
    "state_province",
    "city",
    "street_address",
    "postal_code",
    "phone_number",
    "additional_info",
]

# Super-7 fields
SUPER7_FIELDS = [
    "company_name",
    "trade_style_name",
    "country",
    "street_address",
    "postal_code",
    "city",
    "state_province",
    "website",
    "phone_number",
]

# Source-type weights for scoring
SOURCE_TYPE_WEIGHTS = {
    "government_registry": 1.0,
    "official_website": 0.9,
    "business_directory": 0.75,
    "social_media": 0.7,
    "high_quality_industry_source": 0.75,
    "other": 0.6,
}

# TLD-based URL quality hints
TLD_WEIGHTS = {
    ".gov": 1.0,
    ".gov.in": 1.0,
    ".gov.uk": 1.0,
    ".edu": 0.9,
    ".com": 0.85,
    ".org": 0.82,
    ".net": 0.8,
    ".co": 0.78,
    ".io": 0.75,
}


@dataclass
class AttemptResult:
    attempt_index: int
    strategy_name: str
    raw_object: Dict[str, Any]
    overall_score: float


# ==================== CLIENT ====================

def init_gemini_client() -> genai.Client:
    """
    Initialize Gemini client with Vertex AI.
    Make sure you've run:
        gcloud auth application-default login
    and PROJECT_ID / LOCATION are set correctly.
    """
    return genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)


# ==================== STRING & NAME HELPERS ====================

CORP_SUFFIXES = [
    "inc", "inc.", "llc", "l.l.c", "corp", "corp.", "corporation",
    "ltd", "ltd.", "limited", "pvt", "pvt.", "pvt ltd", "pty", "pty ltd",
    "gmbh", "ag", "sa", "s.a.", "bv", "srl", "s.r.l.",
]


def normalize_whitespace(s: Any) -> str:
    return re.sub(r"\s+", " ", str(s)).strip()


def strip_corp_suffix(name: str) -> str:
    tokens = name.split()
    lowered = [t.lower().strip(",.") for t in tokens]
    end = len(tokens)
    while end > 0 and lowered[end - 1] in CORP_SUFFIXES:
        end -= 1
    return normalize_whitespace(" ".join(tokens[:end]))


def generate_name_variants(company_name: str) -> List[str]:
    """
    Generate cheap but useful variants of the input name.
    """
    if not company_name:
        return []

    base = normalize_whitespace(company_name)
    no_suffix = strip_corp_suffix(base)

    variants = set()
    variants.add(base)
    variants.add(no_suffix)

    tokens = no_suffix.split()
    if not tokens:
        return [base]

    core = tokens[0]

    if len(tokens) >= 2:
        variants.add(" ".join(tokens[:2]))
    if len(tokens) >= 3:
        variants.add(" ".join(tokens[:3]))

    variants.add(core)
    variants.add(f"{core} llc")
    variants.add(f"{core} ltd")
    variants.add(f"{core} corp")

    cleaned = []
    seen = set()
    for v in variants:
        v2 = normalize_whitespace(v)
        low = v2.lower()
        if v2 and low not in seen:
            cleaned.append(v2)
            seen.add(low)

    return cleaned[:10]


def normalize_name_for_similarity(name: str) -> str:
    """
    Aggressive normalization for fuzzy name similarity.
    - lowercasing
    - removing non-alphanumeric
    - removing corp suffixes
    """
    name = name.lower()
    name = re.sub(r"[^a-z0-9\s]", " ", name)
    name = re.sub(r"\s+", " ", name).strip()
    tokens = name.split()
    end = len(tokens)
    while end > 0 and tokens[end - 1] in [s.replace(".", "") for s in CORP_SUFFIXES]:
        end -= 1
    return " ".join(tokens[:end])


def name_similarity(a: str, b: str) -> float:
    """
    Simple token-overlap similarity between two names: Jaccard over tokens.
    Returns 0.0–1.0.
    """
    if not isinstance(a, str) or not isinstance(b, str):
        return 0.0
    a_norm = normalize_name_for_similarity(a)
    b_norm = normalize_name_for_similarity(b)
    if not a_norm or not b_norm:
        return 0.0

    set_a = set(a_norm.split())
    set_b = set(b_norm.split())
    if not set_a or not set_b:
        return 0.0

    inter = len(set_a & set_b)
    union = len(set_a | set_b)
    return inter / union if union else 0.0


# ==================== CONTEXT & STRATEGIES ====================

def build_company_context(row_dict: Dict[str, Any]) -> str:
    lines = []
    for col in DEFAULT_INPUT_COLS:
        v = row_dict.get(col)
        if v is not None and str(v).strip():
            lines.append(f"{col}: {v}")
    return "\n".join(lines)


def build_search_strategies(
    row_dict: Dict[str, Any],
    variants: List[str],
) -> List[Tuple[str, str]]:
    """
    Build textual search strategies (used by the LLM).
    """
    company_name = row_dict.get("company_name", "")
    country = row_dict.get("country", "")
    city = row_dict.get("city", "")
    state = row_dict.get("state_province", "")
    postal = row_dict.get("postal_code", "")

    variants_str = ", ".join(f'"{v}"' for v in variants)
    base_hint = f'Input company name: "{company_name}"'

    strategies: List[Tuple[str, str]] = []

    # Strategy 1: Gov + official website priority
    s1 = f"""
{base_hint}
Use combinations of:
- full & near-full names with "{country}" and "{city} {state} {postal}"
- terms like "business registry", "companies house", "MCA", "secretary of state"
Prioritise:
1) government or official business registries
2) official corporate websites
Use variants: {variants_str}
"""
    strategies.append(("gov_official_strategy", s1.strip()))

    # Strategy 2: Address/geo disambiguation
    s2 = f"""
{base_hint}
Use location as a strong filter:
- queries: "<variant>" "<city>" "<state>" "<postal>" "<country>"
to disambiguate same-named entities.
Use variants: {variants_str}
"""
    strategies.append(("address_geo_strategy", s2.strip()))

    # Strategy 3: Directory + social fallback
    s3 = f"""
{base_hint}
If registry/official results are unclear:
- search business directories like LinkedIn, Bloomberg, Crunchbase, D&B, Manta, Yelp, BBB,
  YellowPages, OpenCorporates, and industry-specific registries
- also company pages on social media if they contain structured business info
Use variants: {variants_str}
"""
    strategies.append(("directory_fallback_strategy", s3.strip()))

    return strategies


# ==================== PROMPT ====================

def build_super7_prompt(
    row_dict: Dict[str, Any],
    strategy_name: str,
    strategy_text: str,
    variants: List[str],
) -> str:
    """
    Main extraction prompt for a single attempt.
    Asks for:
      - "super7": values
      - "sources": per-field metadata
      - "overall_confidence": 0–1
    """
    company_context = build_company_context(row_dict)
    variants_str = ", ".join(f'"{v}"' for v in variants)

    prompt = f"""
You are a reliable company-information extractor using the Google Search tool.

########################
# COMPANY CONTEXT INPUT
########################
{company_context}

########################
# NAME VARIANTS FOR SEARCH
########################
Use these variants in Google Search queries.
All queries must still contain the core token from the original name:
{variants_str}

########################
# SEARCH STRATEGY ({strategy_name})
########################
{strategy_text}

########################
# SOURCE CLASSIFICATION
########################
For each field you extract, you must rely on at least one URL and:
- classify that URL's source_type as one of:
    "government_registry",
    "official_website",
    "business_directory",
    "social_media",
    "high_quality_industry_source",
    "other"
- write a short rationale explaining why that URL supports the field.

########################
# FIELDS TO EXTRACT (Super-7)
########################
Use the best combination of URLs to fill:

- company_name        : official legal or main operating name
- trade_style_name    : trading / doing-business-as name (if any)
- country
- street_address
- postal_code
- city
- state_province
- website             : full URL of official site
- phone_number        : primary business phone

If you cannot confidently determine a field, set its value to null.
For each field above, provide:
- source_url
- source_type
- rationale (short explanation).

Also provide:
- overall_confidence: a number between 0 and 1 describing your confidence
  in the entire Super-7 object.

########################
# OUTPUT FORMAT (STRICT JSON)
########################
Return EXACTLY a single JSON array with ONE object:

[
  {{
    "super7": {{
      "company_name": "...",
      "trade_style_name": "...",
      "country": "...",
      "street_address": "...",
      "postal_code": "...",
      "city": "...",
      "state_province": "...",
      "website": "...",
      "phone_number": "..."
    }},
    "sources": {{
      "company_name": {{
        "source_url": "...",
        "source_type": "government_registry | official_website | business_directory | social_media | high_quality_industry_source | other",
        "rationale": "..."
      }},
      "trade_style_name": {{
        "source_url": "...",
        "source_type": "...",
        "rationale": "..."
      }},
      "country": {{
        "source_url": "...",
        "source_type": "...",
        "rationale": "..."
      }},
      "street_address": {{
        "source_url": "...",
        "source_type": "...",
        "rationale": "..."
      }},
      "postal_code": {{
        "source_url": "...",
        "source_type": "...",
        "rationale": "..."
      }},
      "city": {{
        "source_url": "...",
        "source_type": "...",
        "rationale": "..."
      }},
      "state_province": {{
        "source_url": "...",
        "source_type": "...",
        "rationale": "..."
      }},
      "website": {{
        "source_url": "...",
        "source_type": "...",
        "rationale": "..."
      }},
      "phone_number": {{
        "source_url": "...",
        "source_type": "...",
        "rationale": "..."
      }}
    }},
    "overall_confidence": 0.0
  }}
]

Rules:
- JSON ONLY. No markdown, no commentary outside JSON.
- If a field is null, still provide a sources[field] object with a rationale like "not found" or "not clearly available".
"""
    return prompt.strip()


# ==================== MODEL CALLS ====================

def clean_json_from_text(raw_text: str) -> str:
    """
    Try hard to isolate the JSON part from model output.
    """
    if not raw_text:
        return ""

    text = raw_text.strip()
    text = re.sub(r"^```json", "", text, flags=re.IGNORECASE).strip()
    text = re.sub(r"^```", "", text).strip()
    if "```" in text:
        text = text.split("```", 1)[0].strip()

    idxs = [i for i in (text.find("["), text.find("{")) if i != -1]
    if idxs:
        start = min(idxs)
        text = text[start:].strip()

    return text


def call_gemini_with_grounding(
    client: genai.Client,
    prompt: str,
) -> Optional[Dict[str, Any]]:
    """
    Call Gemini with Google Search grounding and robust JSON parsing.
    """
    config = types.GenerateContentConfig(
        temperature=0.0,
        tools=[types.Tool(google_search=types.GoogleSearch())],
    )

    raw_text = ""
    cleaned = ""
    try:
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=prompt,
            config=config,
        )
        raw_text = response.text or ""
        cleaned = clean_json_from_text(raw_text)

        try:
            data = json.loads(cleaned)
        except json.JSONDecodeError:
            # bracket-slice fallback
            first = cleaned.find("[")
            last = cleaned.rfind("]")
            if first != -1 and last != -1 and last > first:
                candidate = cleaned[first : last + 1]
                data = json.loads(candidate)
            else:
                raise

        if isinstance(data, list) and data:
            return data[0]
        elif isinstance(data, dict):
            return data
        else:
            print("⚠ Unexpected JSON type:", type(data))
            return None

    except json.JSONDecodeError as e:
        print("⚠ JSON decode error:", e)
        print("⚠ Cleaned snippet (first 600 chars):")
        print(cleaned[:600])
        return None

    except Exception as e:
        print("⚠ Error in call_gemini_with_grounding:", e)
        return None


# ==================== SIMPLE URL & CONFIDENCE SCORING ====================

def extract_domain(url: Optional[str]) -> str:
    if not url:
        return ""
    m = re.search(r"https?://([^/]+)", url)
    return m.group(1).lower() if m else ""


def url_quality(url: Optional[str]) -> float:
    if not url:
        return 0.6
    domain = extract_domain(url)
    score = 0.7
    for tld, w in TLD_WEIGHTS.items():
        if domain.endswith(tld):
            score = max(score, w)
    if url.startswith("https://"):
        score += 0.05
    return max(0.4, min(score, 1.0))


def compute_overall_score(obj: Dict[str, Any]) -> float:
    """
    Candidate-level score using:
    - LLM's overall_confidence (0–1)
    - source_type & URL quality for each field
    """
    super7 = obj.get("super7") or {}
    sources = obj.get("sources") or {}
    base_conf = float(obj.get("overall_confidence") or 0.0)

    if not super7 or not sources:
        return 0.0

    per_field_scores = []
    for field in SUPER7_FIELDS:
        meta = sources.get(field) or {}
        src_type = meta.get("source_type", "other")
        src_url = meta.get("source_url")
        src_w = SOURCE_TYPE_WEIGHTS.get(src_type, SOURCE_TYPE_WEIGHTS["other"])
        url_w = url_quality(src_url)

        per_field_scores.append(base_conf * src_w * url_w)

    if not per_field_scores:
        return base_conf

    return round(sum(per_field_scores) / len(per_field_scores), 4)


# ==================== SINGLE-ENTITY RETRY ENGINE ====================

def run_single_attempt(
    client: genai.Client,
    attempt_index: int,
    strategy: Tuple[str, str],
    row_dict: Dict[str, Any],
    variants: List[str],
) -> Optional[AttemptResult]:
    strat_name, strat_text = strategy
    print(f"\n▶ Attempt {attempt_index + 1}: {strat_name}")

    prompt = build_super7_prompt(row_dict, strat_name, strat_text, variants)
    obj = call_gemini_with_grounding(client, prompt)
    if not obj:
        print("  ⚠ Attempt failed (no valid JSON).")
        return None

    overall_score = compute_overall_score(obj)
    print(f"  ✔ Attempt overall candidate score: {overall_score}")
    return AttemptResult(
        attempt_index=attempt_index,
        strategy_name=strat_name,
        raw_object=obj,
        overall_score=overall_score,
    )


def extract_single_entity_for_row(
    client: genai.Client,
    row_dict: Dict[str, Any],
) -> Optional[Dict[str, Any]]:
    """
    Core extractor for a row + its current company_name.
    Uses multi-strategy retry engine, returns the best raw LLM object or None.
    """
    company_name = row_dict.get("company_name", "")
    print("=" * 80)
    print(f"[Entity] Processing name: {company_name}")

    variants = generate_name_variants(company_name)
    strategies = build_search_strategies(row_dict, variants)

    attempts: List[AttemptResult] = []
    print(
        f"=== RETRY ENGINE START (max {MAX_RETRIES} attempts, early-stop at {EARLY_STOP_THRESHOLD}) ==="
    )

    for i in range(min(MAX_RETRIES, len(strategies))):
        res = run_single_attempt(client, i, strategies[i], row_dict, variants)
        if not res:
            continue
        attempts.append(res)
        if res.overall_score >= EARLY_STOP_THRESHOLD:
            print(f"🚀 Early stop at attempt {i + 1} (score {res.overall_score})")
            break

    if not attempts:
        print("❌ All attempts failed for this name.")
        return None

    best = max(attempts, key=lambda a: a.overall_score)
    print(
        f"✅ Best attempt for this name: #{best.attempt_index + 1} "
        f"({best.strategy_name}), score={best.overall_score}"
    )
    return best.raw_object


# ==================== CANDIDATE-ENTITY LAYER ====================

def build_candidate_names(
    original_name: Any,
    primary_obj: Optional[Dict[str, Any]],
) -> List[str]:
    """
    Build a set of candidate names to test:
    - original input name (if string)
    - super7.company_name from primary extraction (if string)
    - super7.trade_style_name from primary extraction (if string)
    plus stripped variations of each.
    """
    candidates: List[str] = []

    if isinstance(original_name, str) and original_name.strip():
        candidates.append(original_name.strip())

    if isinstance(primary_obj, dict):
        super7 = primary_obj.get("super7") or {}
        cname = super7.get("company_name")
        tname = super7.get("trade_style_name")

        if isinstance(cname, str) and cname.strip():
            if cname not in candidates:
                candidates.append(cname.strip())
        if isinstance(tname, str) and tname.strip():
            if tname not in candidates:
                candidates.append(tname.strip())

    expanded: set = set()
    for name in candidates:
        if not isinstance(name, str):
            continue
        expanded.add(name)
        expanded.add(strip_corp_suffix(name))

    final_list: List[str] = []
    seen = set()
    for name in expanded:
        if not isinstance(name, str):
            continue
        name_norm = normalize_whitespace(name)
        low = name_norm.lower()
        if name_norm and low not in seen:
            final_list.append(name_norm)
            seen.add(low)

    return final_list[:4]


def score_candidate_object(
    candidate_obj: Dict[str, Any],
    original_name: str,
) -> float:
    """
    Score a candidate entity using:
    - name similarity between input and candidate.super7.company_name
    - presence of website
    - presence of address/postcode
    - overall_confidence & source-based score from compute_overall_score
    """
    super7 = candidate_obj.get("super7") or {}
    cand_name = super7.get("company_name") or ""

    sim = name_similarity(original_name, cand_name)
    base_score = compute_overall_score(candidate_obj)  # includes model confidence + sources

    # website / address signals
    website = super7.get("website")
    postal = super7.get("postal_code")
    street = super7.get("street_address")

    website_signal = 1.0 if website else 0.0
    address_signal = 1.0 if (postal or street) else 0.0

    final = (
        0.4 * sim +
        0.2 * website_signal +
        0.2 * address_signal +
        0.2 * base_score
    )
    return round(final, 4)


def process_company_row(
    client: genai.Client,
    row: pd.Series,
) -> Dict[str, Any]:
    """
    Full pipeline for one input row:
    1) Primary extraction using the input company_name (multi-strategy, retry).
    2) Build a set of candidate names (original + discovered).
    3) For each candidate name, run a full extraction (all strategies) and score it.
    4) Pick the candidate with the highest score as the final entity.
    5) Return clean JSON: input + super7 + sources + candidate_debug.
    """
    # Defensive: user sometimes passes a full DataFrame by mistake
    if isinstance(row, pd.DataFrame):
        raise ValueError(
            "process_company_row expects a single pandas Series (one row), "
            "but a DataFrame was passed. Use df.iloc[i] for a specific row "
            "or use process_csv_print_only() for the full file."
        )

    if not isinstance(row, pd.Series):
        # Try to coerce to Series if it's a dict-like
        row = pd.Series(dict(row))

    row_dict = row.to_dict()
    original_name = row_dict.get("company_name", "")
    print("=" * 80)
    print(f"🔹 Input row company_name: {original_name}")

    # 1) Primary extraction on original name
    primary_obj = extract_single_entity_for_row(client, row_dict)

    # If even primary failed, bail with placeholder
    if not primary_obj:
        return {
            "input": row_dict,
            "super7": {f: None for f in SUPER7_FIELDS},
            "sources": {
                f: {"source_url": None, "source_type": "other", "rationale": "no valid result"}
                for f in SUPER7_FIELDS
            },
            "candidate_debug": [],
        }

    # 2) Build candidate names
    candidate_names = build_candidate_names(original_name, primary_obj)
    print("\n=== Candidate names to evaluate (full strategies each) ===")
    for cn in candidate_names:
        print(" -", cn)

    candidate_results: List[Tuple[str, Dict[str, Any], float]] = []

    # 3) For each candidate name, run full multi-strategy extraction
    for cand_name in candidate_names:
        cand_row_dict = dict(row_dict)
        cand_row_dict["company_name"] = cand_name
        cand_obj = extract_single_entity_for_row(client, cand_row_dict)
        if not cand_obj:
            continue
        cand_score = score_candidate_object(cand_obj, original_name)
        candidate_results.append((cand_name, cand_obj, cand_score))

    if not candidate_results:
        return {
            "input": row_dict,
            "super7": {f: None for f in SUPER7_FIELDS},
            "sources": {
                f: {"source_url": None, "source_type": "other", "rationale": "no candidates"}
                for f in SUPER7_FIELDS
            },
            "candidate_debug": [],
        }

    # 4) Pick best candidate
    candidate_results.sort(key=lambda t: t[2], reverse=True)
    best_name, best_obj, best_score = candidate_results[0]
    print(f"\n🎯 BEST CANDIDATE: {best_name}  (score={best_score})")

    # Prepare clean super7 + sources
    best_super7 = best_obj.get("super7") or {}
    best_sources = best_obj.get("sources") or {}

    clean_super7 = {f: best_super7.get(f) for f in SUPER7_FIELDS}
    clean_sources = {}
    for f in SUPER7_FIELDS:
        meta = best_sources.get(f) or {}
        clean_sources[f] = {
            "source_url": meta.get("source_url"),
            "source_type": meta.get("source_type"),
            "rationale": meta.get("rationale"),
        }

    candidate_debug = [
        {"candidate_name": name, "score": sc}
        for (name, _obj, sc) in candidate_results
    ]

    return {
        "input": row_dict,
        "super7": clean_super7,
        "sources": clean_sources,
        "candidate_debug": candidate_debug,
    }


# ==================== CSV PIPELINE (PRINT-ONLY) ====================

def process_csv_print_only(input_csv_path: str) -> List[Dict[str, Any]]:
    """
    Process an input CSV and print clean JSON per row.
    """
    df = pd.read_csv(input_csv_path)
    client = init_gemini_client()

    results: List[Dict[str, Any]] = []
    total = len(df)

    for idx, row in df.iterrows():
        print(f"\n\n================= ROW {idx + 1}/{total} =================")
        result = process_company_row(client, row)
        results.append(result)
        print("-" * 80)
        print(f"Result for row {idx} (company: {row.get('company_name', '')})")
        print(json.dumps(result, indent=2, ensure_ascii=False))
        print("-" * 80)

    return results


# ==================== CLI ENTRY (OPTIONAL) ====================

if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser(
        description="Super-7 Vertex AI Extractor (candidate entities, clean output, print-only)."
    )
    parser.add_argument(
        "--input_csv",
        required=True,
        help="Path to input CSV with company_name, country, etc.",
    )
    args = parser.parse_args()

    process_csv_print_only(args.input_csv)


In [ ]:

client = init_gemini_client()

In [ ]:
df = pd.read_csv("/home/gampasa/.ssh/iResearch/i_research_Agent/sample1.csv")
row0 = df.iloc[0]   # pick one row

result = process_company_row(client, row0)
print(json.dumps(result, indent=2, ensure_ascii=False))